# Disciplona: Ciência de Dados

Prof. Luiz Affonso Guedes    - Engenharia de Computação - UFRN 2018-2


## Conteúdo:
- Visualização de mapas
    - Pacote Basemap
- Pacote Seaborn
    - Visualização de dados estatísticos

## Visualização de Mapas
Um tipo comum de visualização na ciência dos dados é o de dados geográficos. A ferramenta principal do Matplotlib para este tipo de visualização é o Basemap toolkit, que é um dos vários kits de ferramentas Matplotlib pertencentes ao mpl_toolkits name space. 

O Basemap parece um pouco complicado de usar e, muitas vezes, mesmo as visualizações simples levam muito mais tempo para renderizar do que você pode esperar. Soluções mais modernas, como a API do Google Maps, podem ser uma escolha melhor para visualizações de mapas mais complexos. 

Ainda assim, o Basemap é uma ferramenta útil para os usuários do Python ter em seus cintos de ferramentas virtuais. 

In [ ]:
# Instalar o basemap no conda (linha de comando)
#! conda install basemap

In [ ]:
# Carregando os pacotes numpy e matplotlib
# carregando o Basemap

%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap

In [ ]:
# Exemplo de mapa mundi simples - só o contorno

map = Basemap()

map.drawcoastlines()

In [ ]:
# Exemplo de mapa mundi

map = Basemap(projection='ortho', 
              lat_0=0, lon_0=0)

#Fill the globe with a blue color 
map.drawmapboundary(fill_color='aqua')
#Fill the continents with the land color
map.fillcontinents(color='coral',lake_color='aqua')

map.drawcoastlines()

In [ ]:
# Com limitação dos países
map = Basemap(projection='ortho', 
              lat_0=0, lon_0=0)

map.drawmapboundary(fill_color='aqua')
map.fillcontinents(color='coral',lake_color='aqua')

map.drawcountries() ## Desenho dos limites dos países


In [ ]:
# Exemplo: mapa mundi

map = Basemap(projection='ortho', 
              lat_0=0, lon_0=0)

map.drawlsmask(land_color = "#ddaa66", 
               ocean_color="#7777ff",
               resolution = 'l')

In [ ]:
# Exemplo de plot do mapa da terra

plt.figure(figsize=(8, 8))
m = Basemap(projection='ortho', resolution=None, lat_0=50, lon_0=-100)
m.bluemarble(scale=0.5)

A coisa útil é que o mundo mostrado aqui não é uma mera imagem. É um funcionamento completo dos eixos Matplotlib que entende coordenadas esféricas e que nos permite sobrepor facilmente dados no mapa! 

Por exemplo, podemos usar uma projeção de mapa diferente, ampliar a América do Norte e traçar a localização de Seattle. Para isto, usaremos uma imagem de etopo (que mostra características topográficas tanto em terra como sob o oceano) como o fundo do mapa:

In [ ]:
# Exemplo de uso do método etopo()

fig = plt.figure(figsize=(8, 8))
m = Basemap(projection='lcc', resolution=None,
            width=8E6, height=8E6, 
            lat_0=45, lon_0=-100,)
m.etopo(scale=0.5, alpha=0.5)

# Mapa (long, lat) para (x, y) no plot
x, y = m(-122.3, 47.6)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, ' Seattle', fontsize=12);



### Projeção de Mapas
A primeira coisa a decidir ao usar mapas é a projeção a ser usada. Você provavelmente está familiarizado com o fato de que é impossível projetar um mapa esférico, como o da Terra, sobre uma superfície plana sem de alguma forma distorcê-lo ou romper sua continuidade. 

Projeções de mapas foram desenvolvidas ao longo da história humana, e há muitas opções! Dependendo do uso pretendido da projeção do mapa, existem determinados recursos do mapa (por exemplo, direção, área, distância, forma ou outras considerações) que são úteis para manter.

O pacote Basemap implementa várias dessas projeções, todas referenciadas por um código de formato curto. Aqui vamos demonstrar brevemente alguns das mais comuns.



Começaremos por definir uma rotina de conveniência para desenhar o nosso mapa mundial juntamente com as linhas de longitude e latitude:

In [ ]:
# Função para desenhar mapas de forma mais conveniente

from itertools import chain

def draw_map(m, scale=0.2):
    # draw a shaded-relief image
    m.shadedrelief(scale=scale)
    
    # lats and longs are returned as a dictionary
    lats = m.drawparallels(np.linspace(-90, 90, 13))
    lons = m.drawmeridians(np.linspace(-180, 180, 13))

    # keys contain the plt.Line2D instances
    lat_lines = chain(*(tup[1][0] for tup in lats.items()))
    lon_lines = chain(*(tup[1][0] for tup in lons.items()))
    all_lines = chain(lat_lines, lon_lines)
    
    # cycle through these lines and set the desired style
    for line in all_lines:
        line.set(linestyle='-', alpha=0.3, color='w')

### Projeções Cilíndricas:
As projeções de mapa mais simples são as projeções cilíndricas, nas quais linhas de latitude e longitude constantes são mapeadas para linhas horizontais e verticais, respectivamente. Este tipo de mapeamento representa bastante bem as regiões equatoriais, mas resulta em distorções extremas perto dos pólos. O espaçamento das linhas de latitude varia entre diferentes projeções cilíndricas, levando a diferentes propriedades de conservação e distorção diferente perto dos pólos. 

Na figura a seguir, mostramos um exemplo da projeção cilíndrica equidistante, que escolhe uma escala de latitude que preserva as distâncias ao longo dos meridianos. Outras projeções cilíndricas são as projeções Mercator (projection = 'merc') e as áreas cilíndricas de área igual (projection = 'cea').

In [ ]:
# Exemplo de projeção simples

map = Basemap(projection='cyl')

map.drawmapboundary(fill_color='aqua')
map.fillcontinents(color='coral',lake_color='aqua')
map.drawcoastlines()

In [ ]:
# Projeções cilíndricas
fig = plt.figure(figsize=(8, 6), edgecolor='w')
m = Basemap(projection='cyl', resolution=None,
            llcrnrlat=-90, urcrnrlat=90,
            llcrnrlon=-180, urcrnrlon=180, )
draw_map(m)

Os argumentos adicionais para Basemap para esta vista especificam a latitude (lat) e longitude (lon) do canto inferior esquerdo (llcrnr -  lower-left corner) e canto superior direito (urcrnr - upper-right corner) para o mapa desejado, em unidades de graus.

### Projeções Pseudo-Cilíndricas
As projeções pseudo-cilíndricas relaxam a exigência de que os meridianos (linhas de longitude constante) permaneçam verticais. Isso pode dar melhores propriedades perto dos pólos da projeção. A projeção Mollweide (projection = 'moll') é um exemplo comum disso, em que todos os meridianos são arcos elípticos. É construído de modo a preservar a área em todo o mapa: embora existam distorções próximas aos pólos, a área de pequenos remendos reflete a área verdadeira. Outras projeções pseudo-cilíndricas são as projeções sinusoidais (projection = 'sinu') e Robinson (projection = 'robin').

In [ ]:
# Exemplo de uso de projeção pseudo-cilíndrica
fig = plt.figure(figsize=(8, 6), edgecolor='w')
m = Basemap(projection='moll', resolution=None,
            lat_0=0, lon_0=0)
draw_map(m)

Altere os valores dos parâmetros lat_0 e lon_0=0 e observe o que ocorre ao gráfico:
lon_0 = 45, 60, 180


In [ ]:
# Exercício


### Projeções de Perspectivas
As projeções de perspectiva são construídas usando uma escolha particular de ponto de perspectiva, semelhante a se fotograr a Terra a partir de um ponto particular do espaço (um ponto que, para algumas projeções, encontra-se tecnicamente dentro da Terra!). 

Um exemplo comum é a projeção ortográfica (projection = 'ortho'), que mostra um lado do globo como visto de um espectador a uma distância muito longa. Como tal, pode mostrar apenas metade do globo de cada vez. Outras projeções baseadas em perspectiva incluem a projeção gnomônica (projection = 'gnom') e projeção estereográfica (projection = 'stere'). Estes são muitas vezes os mais úteis para mostrar pequenas porções do mapa.

In [ ]:
# Exemplo de uso de projeções

map = Basemap(projection='aeqd', lon_0 = 10, lat_0 = 50)

map.drawmapboundary(fill_color='aqua')
map.fillcontinents(color='coral',lake_color='aqua')
map.drawcoastlines()

In [ ]:
# Exemplo de uso de projeções
fig = plt.figure(figsize=(8, 8))
m = Basemap(projection='ortho', resolution=None,
            lat_0=50, lon_0=0)
draw_map(m);

In [ ]:
# Teste o parâmetro projection com 'stere' e 'gnorm'


### Projeções Cônicas
Uma projeção cônica projeta o mapa em um único cone, que é desenrolado. Isso pode levar a propriedades locais muito boas, mas regiões distantes do ponto de foco do cone podem tornar-se muito distorcidas. Um exemplo disto é a projeção da Conica Conformal Lambert (projeção = 'lcc'), que vimos anteriormente no mapa da América do Norte. 

Esse método projeta o mapa em um cone disposto de forma que dois paralelos padrão (especificados em Basemap por lat_1 e lat_2) tenham distâncias bem representadas, com a escala diminuindo entre eles e aumentando para fora deles. 

Outras projeções cônicas úteis são a projeção cônica equidistante (projection = 'eqdc') e a projeção de área igual de Albers (projection = 'aea'). As projeções cônicas, como as projeções em perspectiva, tendem a ser boas escolhas para representar pequenos e médias remendos do globo.

In [ ]:
# Uso de projeção cônica...
fig = plt.figure(figsize=(8, 8))
m = Basemap(projection='lcc', resolution=None,
            lon_0=0, lat_0=50, lat_1=45, lat_2=55,
            width=1.6E7, height=1.2E7)
draw_map(m)

In [ ]:
# Exemplo - projeção da ilha Minorca- Ilhas Baleares - Espanha
map = Basemap(llcrnrlon=3.75,llcrnrlat=39.75,urcrnrlon=4.35,urcrnrlat=40.15, resolution = 'h', epsg=5520)

map.drawmapboundary(fill_color='aqua')
#Fill the continents with the land color
map.fillcontinents(color='coral',lake_color='aqua')
map.drawcoastlines()

In [ ]:
# Exemplo - Ilha Minorca com projeção UTM zone 31N
map = Basemap(llcrnrlon=3.75,llcrnrlat=39.75,urcrnrlon=4.35,urcrnrlat=40.15, epsg=5520)
#http://server.arcgisonline.com/arcgis/rest/services

map.arcgisimage(service='ESRI_Imagery_World_2D', xpixels = 1500, verbose= True)

In [ ]:
# Exemplo: 
map = Basemap(projection='aeqd', lon_0 = 10, lat_0 = 50)

print("Coordenada do ponto no mapa", map(10, 50))
print("posição da coordenada no mapa",  map(20015077.3712, 20015077.3712, inverse=True))

In [ ]:
# Exemplo - mapas das américas do norte e central

map = Basemap(width=12000000,height=9000000,
            rsphere=(6378137.00,6356752.3142),\
            resolution='l',area_thresh=1000.,projection='lcc',\
            lat_1=45.,lat_2=55,lat_0=50,lon_0=-107.)

map.drawmapboundary(fill_color='aqua')
map.fillcontinents(color='#ddaa66', lake_color='aqua')

map.drawcountries()
map.drawstates(color='0.5')

In [ ]:
# Exercício: Desenhe o mapa do Brasil


In [ ]:
# Exercício: Desenhe o mapa do RN


## Usando Mapas com Background
Anteriormente, vimos os métodos bluemarble( ) e shadedrelief( ) para projetar imagens globais no mapa, bem como os métodos drawparallels( ) e drawmeridians( ) para desenhar linhas de latitude e longitude constantes. O pacote Basemap contém uma variedade de funções úteis para desenhar bordas de recursos físicos como continentes, oceanos, lagos e rios, bem como fronteiras políticas, como países e estados e municípios. 


Exemplos de métodos disponíveis no Basemap:

- Physical boundaries and bodies of water
        - drawcoastlines(): Draw continental coast lines
        - drawlsmask(): Draw a mask between the land and sea, for use with projecting images on one or the other
        - drawmapboundary(): Draw the map boundary, including the fill color for oceans.
        - drawrivers(): Draw rivers on the map
        fillcontinents(): Fill the continents with a given color; optionally fill lakes with another color

- Political boundaries
        - drawcountries(): Draw country boundaries
        - drawstates(): Draw US state boundaries
        - drawcounties(): Draw US county boundaries

- Map features
        - drawgreatcircle(): Draw a great circle between two points
        - drawparallels(): Draw lines of constant latitude
        - drawmeridians(): Draw lines of constant longitude
        - drawmapscale(): Draw a linear scale on the map

- Whole-globe images
        - bluemarble(): Project NASA's blue marble image onto the map
        - shadedrelief(): Project a shaded relief image onto the map
        - etopo(): Draw an etopo relief image onto the map
        warpimage(): Project a user-provided image onto the map


## Plotar dados sobre o Mapa
Talvez a peça mais útil do kit de ferramentas Basemap seja a capacidade de superar uma variedade de dados em um plano de fundo. 

As funções do plt funcionam sobre o mapa; você pode usar a instância Basemap para projetar coordenadas de latitude e longitude para coordenadas (x, y) para plotar com plt.

Além disso, existem muitas funções específicas do mapa disponíveis como métodos da instância do Basemap. Estas funcionam de forma muito semelhante às suas equivalentes do padrão Matplotlib, mas possuem um argumento booleano adicional, que, se definido como True, permite que você passe as latitudes e as longitudes cruas para o método, em vez das coordenadas projetadas (x, y).

Alguns métodos específicos para mapas:
    - contour()/contourf() : Desenha linhas de contorno.
    - imshow(): Desenha uma imagem.
    - pcolor()/pcolormesh() : Desenha plot com pseudocolor plot malhas irregulares/regulares.
    - plot(): Desenha linhas ou marcadores.
    - scatter(): Desenha pontos com marcadores.
    - quiver(): Desenha vetores.
    - barbs(): Desenha raw linhas de vento (wind barbs).
    - drawgreatcircle(): Desenha a circle.


### Exemplo sobre análise de dados com mapa do Estado da Califónia

In [ ]:
import pandas as pd
cities = pd.read_csv('https://raw.githubusercontent.com/jakevdp/PythonDataScienceHandbook/master/notebooks/data/california_cities.csv')

# Extração do dada de interesse
lat = cities['latd'].values
lon = cities['longd'].values
population = cities['population_total'].values
area = cities['area_total_km2'].values

In [ ]:
cities.head()

In [ ]:
# 1. Desenhae o mapa em background
fig = plt.figure(figsize=(8, 8))
m = Basemap(projection='lcc', resolution='h', 
            lat_0=37.5, lon_0=-119,
            width=1E6, height=1.2E6)
m.shadedrelief()
m.drawcoastlines(color='gray')
m.drawcountries(color='gray')
m.drawstates(color='gray')

# 2. scatter city data, com cor refletindo a população
# and tamanho (raio) refletindo a área
m.scatter(lon, lat, latlon=True,
          c=np.log10(population), s=area,
          cmap='Reds', alpha=0.5)

# 3. criação da barra de cores e legenda
plt.colorbar(label=r'$\log_{10}({\rm population})$')
plt.clim(3, 7)

# Fazer legenda com pontos (círculos)
for a in [100, 300, 500]:
    plt.scatter([], [], c='k', alpha=0.5, s=a,
                label=str(a) + ' km$^2$')
plt.legend(scatterpoints=1, frameon=False,
           labelspacing=1, loc='lower left')

### Link para Tutorial sobre Basemap
https://basemaptutorial.readthedocs.io/en/latest/index.html

## Pacote Folium para mapas

O pacote Folium fornece um modo fácil para se usar as APIs do Leaflet para programadores de Python.

O Leaflet, por sua vez, é uma biblioteca open-source escrita em JavaScript para desenvolvimento de aplicações com mapas interativos.

Ele também possui funcionalidades de marcadores em Mapas.
 
O pacote Folium suporta vários formatos de dados de mapas, como Image, Video, GeoJSON and TopoJSON. Além disto, o pacote tem uma séria de funcionalidades para OpenStreetMap, Mapbox e Stamen.

Esse pacote permite a ligação de dados a um mapa para visualizações no estilo choropleth, bem como passar marcadores ao mapa.

ver --> 
- geodata br
- mapbox



#### Instalação do Folium

Para se utilizar as funcionalidades do Folium é necessário instalar o pacote no conda:

$ conda install folium -c conda-forge


In [3]:
# Verificando a versão do pacote Folium

import os
import folium

print(folium.__version__)

0.5.0


#### Criação de Mapa

Para se criar uma mapa interativo, basta a chamada da função Map no pacote Folium.

In [3]:
import folium
mapa1 = folium.Map()
mapa1

In [4]:
# Para Salvar o mapa, basta utilizar o método save do objeto mapa.

mapa1.save('index.html')

In [6]:
# Mapa do Brasil
# Utilizar o método Map, com os parâmetros de latitude e longitude
# do Brasil.

latitude = -15.765379
longitude = -47.968776
mapa2 = folium.Map(
    location=[latitude, longitude],
    zoom_start=4)
mapa2

O que faz o parâmetros 'zoom_star'?

Varie esse parâmetro no programa anterior e descubra o seu significado.

In [ ]:
# Exercício: Escreva um programa que mostre o mapa do RN

mapa3 = ????
mapa3

#### Estilo de Mapas

Um outra característica interessante em Folium é que você pode criar diferentes estilos de mapas simplesmente utilizando o parâmetro 'tiles'

O valor default de 'tiles' é o OpenStreetMap, mas é possível utilizar os seguintes padrões, por exemplo:
- Stamen Terrain
- Stamen Toner
- Stamen Watercolor
- Mapbox Bright
- Mapbox Control Room

Para ver todas as opções de parâmetros de configuração pressione (shift+tab) sobre o método Map( ).

In [7]:
# Testando outros estilos de Mapas
#estilo = 'Stamen Toner'
#estilo = 'Stamen Toner'
#estilo = 'Stamen Watercolor'
estilo = 'Mapbox Bright'
#estilo = 'Mapbox Control Room'

mapa4 = folium.Map(
    location = [-15.765379, -47.968776],
    tiles = estilo,
    zoom_start=4
)
mapa4

#### Personalização da folha de estilo

O pacote Folium suporta passagem de qualquer leaflet.js compativel com 'tileset'.

In [7]:
folium.Map(location=[-15.765379, -47.968776],
           zoom_start=6,
           tiles='http://{s}.tiles.yourtiles.com/{z}/{x}/{y}.png',
           attr='Meu Padrão - Personalizado')


O pacote Folium também aceita os 'tileset' personalizados Mapbox. 

Mapbox é a plataforma de dados de localização para aplicações móveis e Web, que fornece blocos de construção para adicionar recursos de localização, como mapas, pesquisa e navegação em qualquer experiência que você criar. 

Para usá-lo, basta passar a sua chave para o folheto personalizado (leaflet) compatível com 'tileset'. 

As seguintes etapas devem ser feitas:

In [9]:
mapa5 = folium.Map(
    location=[-15.765379, -47.968776],
    zoom_start=4,
    tiles='https://api.mapbox.com/styles/v1/mapbox/satellite-v9/tiles/256/{z}/{x}/{y}?access_token=pk.eyJ1IjoibHVpemFmZm9uc28iLCJhIjoiY2o4ZWduMzU3MTIxbjJ3bjhiOTl2N2NrNCJ9.5Sat8JDj-Avd_73e9_OPwg',
    attr='Imagery © <a href="http://mapbox.com">MapBox</a>'
    )

mapa5

In [ ]:
# Exercício
# Crie um Mapa usando o estilo 'Mapbox Satellite Streets'
# Configure a 'image tiles' para 512x512 pixels
# Exiba o Mapa

mapa5 = folium.Map(
    location=[-15.765379, -47.968776],
    zoom_start=12,
    tiles='https://api.mapbox.com/styles/v1/mapbox/outdoors-v10/tiles/256/{z}/{x}/{y}?access_token=your_token',
    attr='Imagery © <a href="http://mapbox.com">MapBox</a>'
    )

mapa5

#### Mapas com Marcadores

Em Folium há várias formas de se sobrepor marcados aos mapas.

O método folium.Marker( ) é usado para inserir marcadores em um mapa.

In [18]:
mapa = folium.Map(
       location=[-5.826592, -35.212558],
       zoom_start=12,
       tiles='Stamen Terrain',
       width='50%',
       height='50%'
       )
folium.Marker([-5.832187, -35.205432], popup='<i>Instituto Metrópole Digital</i>').add_to(mapa)
folium.Marker([-5.842942, -35.198001], popup='<b>Centro Tecnológico</b>').add_to(mapa)

mapa

In [ ]:
# Adicione um marcador sobre a Escola de Ciência e Tecnologia - marcador 'ECT'
# Adicione um marcador sobre o Shopping Middle-Way


In [20]:
# Adicionando cor e tipo de ícone
folium.Marker([-5.832187, -35.205432], 
              popup='<i>Instituto Metrópole Digital</i>',
              icon=folium.Icon(icon='cloud')).add_to(mapa)
folium.Marker([-5.842942, -35.198001], 
              popup='<b>Centro Tecnológico</b>',
              icon=folium.Icon(color='red',
                               icon_color='yellow',
                               icon='info-sign')).add_to(mapa)
mapa

As cores suportadas para os ícones são:
- 'red', 'blue', 'green', 'purple', 'orange', 'darkred','lightred', 'beige', 'darkblue', 'darkgreen', 'cadetblue','darkpurple', 'white', 'pink', 'lightblue', 'lightgreen','gray', 'black', 'lightgray'

No site do 'Font Awesome' (http://fontawesome.io/icons/) é possível encontrar uma infinidade de tipos de ícones. Basta adicionar a informação do prefixo adequado como parâmetro para o método folium.Icon( ).

In [28]:
 # Adicionando cor e tipo de ícone
mapa6 = folium.Map(
       location=[-5.826592, -35.212558],
       zoom_start=12,
       tiles='Stamen Terrain',
       width='50%',
       height='50%'
       )

folium.Marker([-5.842942, -35.198001], 
              popup='<b>Centro Tecnológico</b>',
              icon=folium.Icon(color='red',
                               icon_color='blue',
                               icon='university',
                               prefix = 'fa')).add_to(mapa6)
mapa6

### Clusters de Marcadores
É possível se inserir um grupo de marcadores no mapa. Isto é útil quando há muito marcadores a serem inseridos.

In [32]:
# Exemplo de inserção de marcadores de forma randômica
import numpy as np
from folium import plugins

# tamanho da amostra
N = 100

# exemplos de localizações (latitude e longitude)
dado = np.array(
    [
        np.random.uniform(low=-6.4245, high=-5.2660, size=N),  # lagitude randômica no RN.
        np.random.uniform(low=-38.6200, high=-35.1782, size=N)  # longitude randômica no RN.
    ]
).T

# Criação do Mapa
mapa = folium.Map(
       location=[-5.826592, -35.212558],
       zoom_start=12,
       tiles='Stamen Terrain',
       width='75%',
       height='75%'
       )

# Construção do cluster
plugins.MarkerCluster(dado).add_to(mapa)

mapa

In [33]:
# Exemplo de inserção de marcadores no cluster


# Criação do mapa
mapa = folium.Map(
    location=[-5.826592, -35.212558],
    zoom_start=12,
    tiles='Stamen Terrain',
    width='75%',
    height='75%'
)

# Criação do cluster de pontos (sem nenhum marcador) e associação dele a um mapa

marker_cluster = plugins.MarkerCluster().add_to(mapa)


# Associação de dois marcadores ao cluster já criado

folium.Marker([-5.832187, -35.205432], popup='<i>Instituto Metrópole Digital</i>').add_to(marker_cluster)
folium.Marker([-5.842942, -35.198001], popup='<b>Centro Tecnológico</b>').add_to(marker_cluster)

mapa



In [ ]:
# Exercício
# Crie um mapa com marcadores dos 10 hoteis em Natal-RN mais bem avaliados,
# segundo o site do Decolar.com (www.decolar.com)
# Escolha cores e ícones adequados para os marcadores
# Agrupe os Hoteis por região da cidade - caso haja hoteis em mais de uma região
 


#### Mapas de Calor
Para criar mapas de color (HeatMaps) é necessário importar o plug-in específico do Folium. Depis é só utilizar a função HeatMap( ) sobre os dados de marcadores.


In [35]:
from folium.plugins import HeatMap
from folium import plugins
import numpy as np

# tamanho da amostra de pontos
N = 100

# latitudes e longitudes das amostras - dispostas randomicamente
data = np.array(
    [
        np.random.uniform(low=-6.4245, high=-5.2660, size=N),  # latitudes.
        np.random.uniform(low=-38.6200, high=-35.1782, size=N)  # longitudes.
    ]
).T.tolist()  # necessidade de se transformar para tipo list

# Criação do Mapa
m = folium.Map( 
    location=[-5.826592, -35.212558],
    zoom_start=12,
    tiles='Stamen Terrain',
    width='75%',
    height='75%'
)

# data : lista de pontos no formato [lat, lng] OU [lat, lng, weight]
HeatMap(data).add_to(m)

m

In [40]:
# Função para habilitar PopUps
# lat/lng de popovers

m = folium.Map(
    location=[-5.826592, -35.212558],
    zoom_start=12,
    tiles='Stamen Terrain',
    width='50%',
    height='50%'
)

m.add_child(folium.LatLngPopup())
m



#### Vega popup

In [43]:
import json
import numpy as np
import vincent

# Dictionary is the dataset
scatter_points = {
    'x': np.random.uniform(size=(100,)),
    'y': np.random.uniform(size=(100,)),
}

# Let's create the vincent chart.
scatter_chart = vincent.Scatter(scatter_points,
                                iter_idx='x',
                                width=600,
                                height=300)

# Create a map
m = folium.Map(
    location=[-5.826592, -35.212558],
    zoom_start=12,
    width='50%',
    height='50%',
    tiles='Stamen Terrain',
)

# Create an object popup and adding a graph for it
popup = folium.Popup(max_width=650)
folium.Vega(scatter_chart, height=350, width=650).add_to(popup)

# Print a icon on map
folium.Marker([-5.832187, -35.205432], 
              icon=folium.Icon(icon='cloud'),
              popup=popup).add_to(m)
m

### PopUps Pandas

In [41]:
# Exemplo de uso d PopUps com DataFrames Pandas

import pandas as pd

# Create a dataframe
df = pd.DataFrame(data=[['2016', 'A'], ['2017', 'A+']], columns=['Year', 'Grade'])

# Create a map
m = folium.Map(
    location=[-5.826592, -35.212558],
    zoom_start=12,
    width='50%',
    height='50%',
    tiles='Stamen Terrain',
)

# Render a DataFrame as an HTML table.
html = df.to_html(classes='table table-striped table-hover table-condensed table-responsive')

# Create a popup
popup = folium.Popup(html)

# Print a icon on map
folium.Marker([-5.832187, -35.205432], 
              icon=folium.Icon(icon='cloud'),
              popup=popup).add_to(m)
m

